In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hnswlib
from sklearn.neighbors import NearestNeighbors
from heapq import heappush, heappop

In [2]:

# Path to your GloVe file (update this based on your downloaded version)
glove_path = '/Users/tanishqchaudhari/Desktop/DataScience Proj  datasets/Dataset/glove.6B.100d.txt'

# Load GloVe vectors
word_to_vec = {}
words = []
vectors = []

with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # First token is the word
        vector = np.array(values[1:], dtype=np.float32)  # Rest are vector values
        word_to_vec[word] = vector
        words.append(word)
        vectors.append(vector)

# Convert to numpy array
vectors = np.array(vectors, dtype=np.float32)
print(f"Loaded {len(words)} word vectors of dimension {vectors.shape[1]}")

Loaded 400000 word vectors of dimension 100


In [3]:
print(vectors.shape)

(400000, 100)


In [4]:
import numpy as np
import time
import logging
import random
import math
from heapq import heappush, heappop
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [5]:
import numpy as np

def precompute_all_query_pcas(pca_models, pca_applied_layers, query_vectors):
    """
    Precomputes PCA projections for a batch of query vectors.
    Returns a list where each index corresponds to the layer number, and contains
    either the PCA-transformed queries (np.ndarray) or None if not used.
    """
    max_layer = max(pca_applied_layers) + 1 if pca_applied_layers else 0
    pca_queries = [None] * max_layer  # indexed list for O(1) access

    query_vectors_np = np.asarray(query_vectors)

    for l_idx in pca_applied_layers:
        model = pca_models[l_idx]  # direct index, avoid .get()
        pca_queries[l_idx] = model.transform(query_vectors_np)

    return pca_queries


In [6]:
def precompute_all_query_pcas(pca_models, pca_applied_layers, query_vectors):
    """
    Precomputes PCA projections for a batch of query vectors.
    Returns a list where each index = layer number, and value is
    either the (n_queries, pca_components) array or None.
    """
    if not pca_applied_layers:
        return []

    max_layer = max(pca_applied_layers) + 1
    pca_queries = [None] * max_layer
    Q = np.asarray(query_vectors)

    for l in pca_applied_layers:
        model = pca_models[l]
        pca_queries[l] = model.transform(Q)

    return pca_queries

In [7]:
import math
import random
import logging
import numpy as np
from sklearn.decomposition import PCA
from heapq import heappush, heappop
import time # Added for potential timing within methods if needed

logger = logging.getLogger(__name__)

# --- precompute_all_query_pcas function remains the same ---
def precompute_all_query_pcas(pca_models, pca_applied_layers, query_vectors):
    """
    Precomputes PCA projections for a batch of query vectors.
    Returns a list where each index = layer number, and value is
    either the (n_queries, pca_components) array or None.
    """
    if not pca_applied_layers:
        return []

    max_layer = max(pca_applied_layers) + 1
    pca_queries = [None] * max_layer
    Q = np.asarray(query_vectors)

    for l in pca_applied_layers:
        # Check if the model exists for the layer (robustness)
        if l in pca_models:
            model = pca_models[l]
            pca_queries[l] = model.transform(Q)
        # else: # Should not happen if pca_applied_layers is consistent
        #    logger.warning(f"Model for supposedly applied PCA layer {l} not found during query precomputation.")


    return pca_queries

# --- Modified optHNSWPCA Class ---
class optHNSWPCA:
    """
    HNSW with optional PCA-based acceleration, applied only to layers >= 2.
    """
    def __init__(self, dim, max_elements, M=16, ef_construction=200,
                 pca_yes=False, pca_components=50): # Removed pca_top_layers
        self.dim = dim
        self.max_elements = max_elements
        self.M = M
        self.ef_construction = ef_construction
        self.pca_yes = pca_yes and pca_components < dim
        # pca_top_layers parameter is removed
        self.pca_components = min(pca_components, dim)

        self.vectors = [] # List to store full-dimensional vectors
        self.layers = [] # List of dictionaries representing graph layers
        self.entry_point = None # Index of the entry point node
        self.entry_point_level = -1 # Highest layer the entry point exists in

        # PCA internals - populated by finalize_pca
        self.pca_models = {} # Dictionary {layer_idx: fitted_pca_model}
        self.pca_applied_layers = set() # Set of layer indices where PCA is active

        # Optimized lookup structures for PCA search - populated by finalize_pca
        self._rv_arr = None      # Numpy array shape (max_elements, pca_components) storing reduced vectors
        self._rv_mask = None     # Boolean mask (max_elements,) indicating if a node has a reduced vector
        self._use_pca_layer = None  # Boolean mask (num_layers,) indicating if PCA logic should be used for a layer

        logger.info(f"Initialized HNSW (dim={dim}, PCA={'on (layers >= 2)' if self.pca_yes else 'off'}, PCA components={self.pca_components if self.pca_yes else 'N/A'})")

    def _get_layer(self):
        # Determine the layer for a new node using exponential decay based on M
        ml = 1 / math.log(self.M) if self.M > 1 else 1
        # Ensure level is at least 0
        return max(0, int(-math.log(random.random()) * ml))

    def _distance(self, idx1, idx2):
        # Calculate Euclidean distance between two vectors using their indices
        # Assumes indices are valid and vectors exist
        return np.linalg.norm(self.vectors[idx1] - self.vectors[idx2])

    def _search_layer_standard(self, query_vec, layer_idx, ep_idx, ef):
        # Standard HNSW search within a single layer using full-dimensional vectors
        graph = self.layers[layer_idx]
        if not graph: # Layer is empty
            return []

        # Ensure the entry point exists in this layer's graph
        if ep_idx not in graph:
            # If ep_idx isn't valid, try starting from the first node in the layer
            try:
                ep_idx = next(iter(graph))
            except StopIteration: # Layer exists but is empty (shouldn't happen if !graph check passes)
                 return []

        visited = {ep_idx} # Keep track of visited nodes to avoid cycles/redundancy
        dist0 = np.linalg.norm(query_vec - self.vectors[ep_idx]) # Distance to entry point
        candidates = [(dist0, ep_idx)] # Min-heap storing (distance, node_idx) candidates to visit
        results = [(-dist0, ep_idx)] # Max-heap storing (-distance, node_idx) of potential neighbors found

        while candidates:
            dist, cur = heappop(candidates) # Get the closest candidate node
            farthest_dist_in_results = -results[0][0] # Distance to the farthest node currently in results

            # Optimization: If the closest candidate is farther than the farthest result
            # and we already have enough results (ef), we can stop exploring this path.
            if dist > farthest_dist_in_results and len(results) >= ef:
                break

            # Explore neighbors of the current node
            # Use .get(cur, []) to handle nodes that might have no outgoing links (shouldn't happen with bidirectional links)
            for nb in graph.get(cur, []):
                if nb in visited:
                    continue # Skip already visited nodes
                visited.add(nb)

                # Check if neighbor index is valid (robustness for potential graph inconsistencies)
                if nb >= len(self.vectors):
                    logger.warning(f"Neighbor index {nb} out of bounds (max: {len(self.vectors)-1}) in layer {layer_idx}. Skipping.")
                    continue

                d = np.linalg.norm(query_vec - self.vectors[nb]) # Calculate distance to neighbor

                # If the neighbor is closer than the farthest result, or we don't have ef results yet
                if d < farthest_dist_in_results or len(results) < ef:
                    heappush(results, (-d, nb)) # Add to results (max-heap, so use negative distance)
                    # If results exceed ef, remove the farthest one
                    if len(results) > ef:
                        heappop(results)
                    # Add the neighbor to candidates for further exploration
                    heappush(candidates, (d, nb))

        # Return the top 'ef' results, sorted by actual distance (ascending)
        return sorted([(-d, idx) for d, idx in results])[:ef]

    def insert(self, vector):
        # Insert a new vector into the HNSW graph
        vec = np.asarray(vector)
        idx = len(self.vectors) # Index for the new vector

        if idx >= self.max_elements:
            logger.error(f"Attempted to insert vector {idx}, but index capacity ({self.max_elements}) reached.")
            raise MemoryError("Index full")

        self.vectors.append(vec) # Store the full-dimensional vector
        level = self._get_layer() # Determine the maximum layer this node will exist in

        # Ensure the layers list is large enough
        while len(self.layers) <= level:
            self.layers.append({}) # Add new empty layers if needed

        ep = self.entry_point # Start search from the global entry point

        # Phase 1: Find entry points for each layer from top down to target 'level' + 1
        # This navigates the upper layers to find a good starting point for insertion in the layers below.
        for l in range(self.entry_point_level, level, -1): # Iterate downwards from highest layer down to level+1
            if ep is None: break # Should not happen if entry_point_level >= 0
            if not self.layers[l]: continue # Skip empty layers

            # Search in layer 'l' to find the closest node to the new vector 'vec'
            # Use ef=1 as we only need the single best entry point for the next layer down.
            res = self._search_layer_standard(vec, l, ep, ef=1)
            if res:
                # Update entry point for the next lower layer search
                ep = res[0][1]
            # else: # No node found in this layer? Continue with the current ep.

        # Phase 2: Insert the node into layers from 'level' down to 0
        # At each layer, find neighbors and establish connections.
        for l in range(min(level, self.entry_point_level if self.entry_point is not None else -1), -1, -1):
            graph = self.layers[l]

            # If entry point 'ep' is None (e.g., first node) and graph isn't empty, pick a random node as ep
            if ep is None and graph:
                 try: ep = next(iter(graph))
                 except StopIteration: pass # Graph became empty?

            neigh = [] # List to store neighbors found in this layer
            if ep is not None:
                # Search for ef_construction nearest neighbors to 'vec' in layer 'l', starting from 'ep'
                neigh = self._search_layer_standard(vec, l, ep, self.ef_construction)
                if neigh:
                    # Update the entry point 'ep' for the next layer down (layer l-1)
                    # The closest node found in this layer is a good starting point for the layer below.
                    ep = neigh[0][1]
                # else: ep remains the same if no neighbors found

            # Select the top M neighbors based on distance
            conns = [nid for _, nid in neigh[:self.M]]
            # Add outgoing connections from the new node 'idx' to its neighbors
            graph[idx] = conns

            # Add incoming connections from neighbors back to the new node 'idx'
            for nb in conns:
                # Ensure neighbor exists in the graph dict
                graph.setdefault(nb, [])
                neighbor_conns = graph[nb]
                neighbor_conns.append(idx)

                # Pruning: If neighbor now has > M connections, remove the farthest one
                if len(neighbor_conns) > self.M:
                    dists = [(self._distance(nb, c), c) for c in neighbor_conns]
                    dists.sort() # Sort by distance
                    # Keep only the M closest connections
                    graph[nb] = [c for _, c in dists[:self.M]]

        # Update global entry point if the new node is inserted at a higher level than the current entry point
        if self.entry_point is None or level > self.entry_point_level:
            self.entry_point = idx
            self.entry_point_level = level


    def finalize_pca(self):
        # Fit PCA models for specified layers and prepare optimized structures for PCA search.
        if not self.pca_yes:
            logger.info("PCA is disabled (pca_yes=False). Skipping PCA finalization.")
            return

        total_layers = len(self.layers)
        self.pca_models.clear()
        self.pca_applied_layers.clear()

        # --- Define the starting layer for PCA ---
        # Apply PCA only to layers 2 and above.
        start_pca_consideration_layer = 2

        logger.info(f"Attempting to apply PCA (components={self.pca_components}) to layers >= {start_pca_consideration_layer}...")

        layers_fitted = []
        # Iterate through layers starting from layer 2 up to the highest layer
        for l in range(start_pca_consideration_layer, total_layers):
            if l not in self.layers: # Should not happen, but defensive check
                continue

            graph = self.layers[l]
            # Get valid indices of nodes present in this layer's graph
            idxs = [i for i in graph if i < len(self.vectors)] # Ensure index is within bounds of stored vectors

            # Check if there are enough nodes in the layer to perform PCA
            if len(idxs) <= self.pca_components:
                logger.info(f"Skipping PCA on layer {l}: Not enough nodes ({len(idxs)}) for {self.pca_components} components.")
                continue

            logger.info(f"Fitting PCA for layer {l} on {len(idxs)} nodes...")
            try:
                # Stack the full-dimensional vectors for nodes in this layer
                arr = np.vstack([self.vectors[i] for i in idxs])

                # Fit PCA model
                pca = PCA(n_components=self.pca_components)
                pca.fit(arr)

                # Store the fitted model and mark the layer as having PCA applied
                self.pca_models[l] = pca
                self.pca_applied_layers.add(l)
                layers_fitted.append(l)
                logger.info(f"PCA successfully fitted for layer {l}.")

            except Exception as e:
                logger.error(f"Error applying PCA to layer {l}: {e}. Skipping PCA for this layer.")
                # Ensure layer is not marked if PCA failed
                if l in self.pca_applied_layers:
                     self.pca_applied_layers.remove(l)
                if l in self.pca_models:
                     del self.pca_models[l]
                continue

        # If no layers ended up having PCA applied, log and exit finalize_pca
        if not self.pca_applied_layers:
             logger.warning("PCA was enabled, but no layers met the criteria (layer >= 2 and sufficient nodes). PCA search will not be used.")
             # Initialize mask for search function robustness, although it won't be used
             self._use_pca_layer = np.zeros(total_layers, dtype=bool)
             return

        # --- Prepare optimized structures only if PCA was applied to at least one layer ---
        logger.info(f"Preparing optimized PCA lookup structures for layers: {sorted(list(self.pca_applied_layers))}")

        # Allocate arrays for reduced vectors and masks
        # Using float32 for reduced vectors can save memory
        self._rv_arr = np.zeros((self.max_elements, self.pca_components), dtype=np.float32)
        self._rv_mask = np.zeros(self.max_elements, dtype=bool) # Tracks which nodes have a reduced vector stored
        self._use_pca_layer = np.zeros(total_layers, dtype=bool) # Tracks which layers use PCA search logic

        # Fill the reduced vectors array and update masks
        for l in self.pca_applied_layers:
            # Get node indices again for this layer
            idxs = [i for i in self.layers[l] if i < len(self.vectors)]
            if not idxs: continue # Skip if layer somehow became empty

            # Get the corresponding PCA model
            pca_model = self.pca_models[l]

            # Stack vectors and transform them using the layer's PCA model
            # Note: It might be slightly more efficient to transform only vectors
            # not already transformed by a higher layer's PCA, but this is simpler.
            arr = np.vstack([self.vectors[i] for i in idxs])
            red = pca_model.transform(arr).astype(np.float32) # Transform and cast to float32

            # Store the reduced vectors in the pre-allocated array and set the mask
            for node_idx, vec_red in zip(idxs, red):
                if node_idx < self.max_elements: # Bounds check
                    self._rv_arr[node_idx] = vec_red
                    self._rv_mask[node_idx] = True # Mark this node as having a reduced vector
                else:
                     logger.warning(f"Node index {node_idx} >= max_elements ({self.max_elements}) during PCA finalization. Skipping.")


            self._use_pca_layer[l] = True # Mark this layer for using PCA search logic

        logger.info(f"PCA finalization complete. Applied to layers: {sorted(list(self.pca_applied_layers))}")


    def _search_layer_pca_precomputed(self, query_idx, query_vec, pca_queries, layer_idx, ep_idx, ef):
        # PCA-accelerated search within a single layer using precomputed PCA-transformed queries
        graph = self.layers[layer_idx]
        if not graph: return []

        # Check if the precomputed PCA query exists for this layer and query index
        if pca_queries[layer_idx] is None or query_idx >= len(pca_queries[layer_idx]):
             logger.warning(f"PCA query vector missing for query {query_idx} in layer {layer_idx}. Falling back to standard search for this layer.")
             # Fallback to standard search for this specific layer call
             return self._search_layer_standard(query_vec, layer_idx, ep_idx, ef)

        q_red = pca_queries[layer_idx][query_idx] # Get the precomputed reduced query vector

        # Ensure entry point exists and has a reduced vector
        if ep_idx not in graph or not self._rv_mask[ep_idx]:
             logger.warning(f"Entry point {ep_idx} missing or lacks reduced vector in PCA layer {layer_idx}. Trying graph iterator.")
             try:
                 # Try to find a valid starting node in the graph that *does* have a reduced vector
                 valid_eps = [idx for idx in graph if self._rv_mask[idx]]
                 if not valid_eps:
                      logger.error(f"No nodes with reduced vectors found in PCA layer {layer_idx}. Cannot start PCA search. Falling back.")
                      return self._search_layer_standard(query_vec, layer_idx, ep_idx if ep_idx in graph else next(iter(graph)), ef)
                 ep_idx = random.choice(valid_eps) # Start from a random valid node
                 logger.info(f"Restarting PCA search in layer {layer_idx} from new entry point {ep_idx}.")
             except StopIteration:
                 logger.error(f"Graph iterator failed in PCA layer {layer_idx} fallback. Returning empty.")
                 return [] # Layer is effectively empty or unusable

        visited = {ep_idx}
        # Calculate initial distance using reduced vectors
        init_dist = np.linalg.norm(q_red - self._rv_arr[ep_idx])
        candidates = [(init_dist, ep_idx)] # Min-heap for candidates
        results = [(-init_dist, ep_idx)] # Max-heap for results

        while candidates:
            dist, cur = heappop(candidates)
            farthest_dist_in_results = -results[0][0]

            if dist > farthest_dist_in_results and len(results) >= ef:
                break

            for nb in graph.get(cur, []):
                if nb in visited:
                    continue
                visited.add(nb)

                # Check if neighbor index is valid (bounds check)
                if nb >= self.max_elements:
                     logger.warning(f"Neighbor index {nb} out of bounds ({self.max_elements}) in PCA layer {layer_idx} search. Skipping.")
                     continue

                # Check if the neighbor has a precomputed reduced vector
                if self._rv_mask[nb]:
                    # Calculate distance in reduced PCA space
                    d = np.linalg.norm(q_red - self._rv_arr[nb])
                else:
                    # Fallback: Neighbor exists in this layer but somehow doesn't have a reduced vector
                    # This shouldn't happen if finalize_pca worked correctly for all nodes in the layer.
                    # Calculate distance using original vectors as a fallback.
                    logger.warning(f"Node {nb} in PCA layer {layer_idx} missing reduced vector. Using full distance calculation.")
                    # Ensure the original vector exists
                    if nb >= len(self.vectors):
                         logger.error(f"Node {nb} index out of bounds for original vectors too. Skipping.")
                         continue
                    d = np.linalg.norm(query_vec - self.vectors[nb])

                # Add to results/candidates if it's a potential neighbor
                if d < farthest_dist_in_results or len(results) < ef:
                    heappush(results, (-d, nb))
                    if len(results) > ef:
                        heappop(results)
                    heappush(candidates, (d, nb))

        # Return results sorted by distance
        return sorted([(-d, idx) for d, idx in results])[:ef]


    def search(self, query_vec, k=10):
        # Standard HNSW search (no PCA acceleration used)
        ef = max(self.ef_construction, k) # Use ef_construction or k, whichever is larger, for search quality
        ep = self.entry_point
        if ep is None: # Handle empty index
            return []

        q = np.asarray(query_vec) # Ensure query is a numpy array

        current_ep_level = self.entry_point_level
        # Phase 1: Navigate top layers (down to layer 1) to find entry point for layer 0
        for l in range(current_ep_level, 0, -1): # Iterate from highest layer down to layer 1
             if not self.layers[l]: continue # Skip empty layers

             # Find the single closest node in layer 'l' to use as entry point for layer 'l-1'
             res = self._search_layer_standard(q, l, ep, ef=1)
             if res:
                 ep = res[0][1] # Update entry point for the next layer down

        # Phase 2: Perform exhaustive search in the base layer (layer 0)
        # Use the entry point 'ep' found from navigating upper layers.
        # Use the larger 'ef' value for higher recall in the base layer.
        res = self._search_layer_standard(q, 0, ep, ef)

        # Return the indices of the top k results
        return [i for _, i in res[:k]]

    def search_pca_opt(self, query_idx, query_vec, pca_queries, k=10):
        # HNSW search potentially accelerated by PCA in layers >= 2.
        ef = max(self.ef_construction, k)
        ep = self.entry_point
        if ep is None: return []

        q = np.asarray(query_vec) # Original query vector (needed for fallbacks)

        current_ep_level = self.entry_point_level

        # Check if PCA structures are initialized (robustness)
        pca_ready = self.pca_yes and self._use_pca_layer is not None and pca_queries is not None

        # Phase 1: Navigate top layers (down to layer 1)
        for l in range(current_ep_level, 0, -1):
            if not self.layers[l]: continue

            use_pca_for_this_layer = False
            # Determine if PCA should be used for this layer's search step
            if pca_ready and l < len(self._use_pca_layer) and self._use_pca_layer[l] and l < len(pca_queries):
                  use_pca_for_this_layer = True

            # Perform search in layer 'l' (either PCA or standard) to find entry point for 'l-1'
            if use_pca_for_this_layer:
                 # Use PCA search, ef=1 as we only need the best entry point
                 res = self._search_layer_pca_precomputed(query_idx, q, pca_queries, l, ep, ef=1)
            else:
                 # Use standard search, ef=1
                 res = self._search_layer_standard(q, l, ep, ef=1)

            if res:
                ep = res[0][1] # Update entry point for next layer down

        # Phase 2: Search in base layer (layer 0) - *Always use standard search*
        # Layer 0 is intentionally excluded from PCA application in finalize_pca.
        # We use the larger 'ef' here for quality.
        res = self._search_layer_standard(q, 0, ep, ef)

        # Return top k results
        return [i for (_, i) in res[:k]]

In [8]:
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import logging # Make sure logging is configured

# Configure logging if not already done
logging.basicConfig(level=logging.INFO)

# Assume 'vectors' is pre-loaded (e.g., from GloVe)
# Example Placeholder:
# NUM_ELEMENTS = 400000
# DIMENSION = 100
# vectors = np.random.rand(NUM_ELEMENTS, DIMENSION).astype(np.float32)


if __name__ == "__main__":
    # --- parameters ---
    num_vectors = vectors.shape[0] # Use actual shape
    num_queries = 40_000
    dim = vectors.shape[1]         # Use actual shape
    k = 100
    M = 20
    ef_construction = 300
    # pca_top_layers REMOVED from parameters here
    pca_components = 25 # Example: Use a less aggressive value

    # load or generate data
    print("Selecting query data...")
    # Ensure num_queries isn't larger than num_vectors
    actual_num_queries = min(num_queries, num_vectors)
    query_indices = np.random.choice(num_vectors, actual_num_queries, replace=False)
    queries = vectors[query_indices]
    print(f"Using {actual_num_queries} queries.")


    # compute ground truth once
    print("Computing ground truth...")
    t_gt0 = time.perf_counter()
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
    nn.fit(vectors) # Use the full dataset 'vectors'
    gt_d, gt_i = nn.kneighbors(queries)
    t_gt1 = time.perf_counter()
    print(f"Ground truth computed in {t_gt1 - t_gt0:.2f}s")


    # --- Build standard (non-PCA) index ---
    print("\nBuilding standard HNSW index (no PCA)...")
    idx_std = optHNSWPCA(
        dim, num_vectors + 10, M, ef_construction, # Add buffer to max_elements
        pca_yes=False,
        # No pca_top_layers argument needed
        pca_components=pca_components # Pass components even if pca_yes=False (ignored)
    )
    t0 = time.perf_counter()
    for v in tqdm(vectors, desc="Indexing standard HNSW"): # Iterate over 'vectors'
        idx_std.insert(v)
    build_std = time.perf_counter() - t0
    print(f"Standard build time: {build_std:.2f}s")
    print(f"Standard index height: {idx_std.entry_point_level}")


    # --- Build PCA-enabled index ---
    print("\nBuilding PCA-enabled HNSW index...")
    idx_pca = optHNSWPCA(
        dim, num_vectors + 10, M, ef_construction, # Add buffer
        pca_yes=True,
        # No pca_top_layers argument needed
        pca_components=pca_components # Specify desired components
    )
    t0 = time.perf_counter()
    # Build the PCA index using the *same* data
    for v in tqdm(vectors, desc="Indexing PCA-enabled HNSW"): # Iterate over 'vectors'
        idx_pca.insert(v)
    build_pca = time.perf_counter() - t0
    print(f"PCA-enabled index build time (before finalize): {build_pca:.2f}s")
    print(f"PCA-enabled index height: {idx_pca.entry_point_level}")


    # finalize PCA on PCA-enabled index
    print("\nFinalizing PCA models on index...")
    t0 = time.perf_counter()
    idx_pca.finalize_pca() # This now applies PCA only to layers >= 2
    fit_pca = time.perf_counter() - t0
    print(f"PCA finalize time: {fit_pca:.2f}s")

    # Precompute PCA projections for queries for the PCA-enabled index
    print("\nPrecomputing PCA projections for queries...")
    t0 = time.perf_counter()
    # Ensure pca_models and pca_applied_layers exist before calling
    pca_q = []
    if idx_pca.pca_yes and idx_pca.pca_applied_layers:
         pca_q = precompute_all_query_pcas(idx_pca.pca_models, idx_pca.pca_applied_layers, queries)
    else:
         print("Skipping query PCA precomputation as PCA was not applied to any layers.")
    t_pca_q = time.perf_counter()-t0
    print(f"Query PCA precomputation time: {t_pca_q:.4f}s")


    # helper for recall calculation
    def recall_at_k(ground_truth_indices, predicted_indices, k_val):
        """ Calculates recall for a single query """
        # Ensure predicted_indices is iterable and not None
        if predicted_indices is None:
             predicted_indices = []
        return len(set(ground_truth_indices[:k_val]) & set(predicted_indices[:k_val])) / k_val

    # --- Evaluate standard search ---
    print("\nEvaluating standard search...")
    results_std = [None] * actual_num_queries
    times_std = []
    total_r_std = 0
    tstart_query_std = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="Standard query")):
        ts = time.perf_counter()
        results_std[i] = idx_std.search(q, k) # Store results
        te = time.perf_counter()
        times_std.append(te - ts)
        total_r_std += recall_at_k(gt_i[i], results_std[i], k) # Use the helper
    tend_query_std = time.perf_counter()
    tot_time_std = tend_query_std - tstart_query_std
    avg_time_std = np.mean(times_std) * 1000 if times_std else 0
    recall_std = total_r_std / actual_num_queries if actual_num_queries > 0 else 0


    # --- Evaluate PCA-optimized search ---
    print("\nEvaluating PCA-optimized search...")
    results_pca = [None] * actual_num_queries
    times_pca = []
    total_r_pca = 0
    tstart_query_pca = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="PCA-opt query")):
        ts = time.perf_counter()
        # Call search_pca_opt, passing the query index 'i' and original query 'q'
        results_pca[i] = idx_pca.search_pca_opt(i, q, pca_q, k) # Store results
        te = time.perf_counter()
        times_pca.append(te - ts)
        # Calculate recall using ground truth for query i and predictions for query i
        total_r_pca += recall_at_k(gt_i[i], results_pca[i], k) # Use the helper
    tend_query_pca = time.perf_counter()
    tot_time_pca = tend_query_pca - tstart_query_pca
    avg_time_pca = np.mean(times_pca) * 1000 if times_pca else 0
    recall_pca = total_r_pca / actual_num_queries if actual_num_queries > 0 else 0


    # --- Output results ---
    print("\n--- Results ---")
    print(f"Parameters: M={M}, ef_construction={ef_construction}, k={k}")
    if idx_pca.pca_yes:
        print(f"PCA Parameters: components={pca_components}, applied_to_layers={sorted(list(idx_pca.pca_applied_layers)) if idx_pca.pca_applied_layers else 'None'}")
    else:
         print("PCA Parameters: PCA Disabled")

    print("\nBuild Times:")
    print(f"Standard Build: {build_std:.2f}s")
    print(f"PCA Build (Index): {build_pca:.2f}s")
    print(f"PCA Build (Finalize): {fit_pca:.2f}s")
    print(f"Total PCA Build: {build_pca + fit_pca:.2f}s")


    print(f"\nStandard: recall@{k}={recall_std:.4f}, total_query_time={tot_time_std:.2f}s, avg_query_time={avg_time_std:.2f}ms")
    print(f"PCA-opt:  recall@{k}={recall_pca:.4f}, total_query_time={tot_time_pca:.2f}s, avg_query_time={avg_time_pca:.2f}ms")

    print("\n--- Comparison Summary ---")
    print(f"Method         | Recall@{k} | TotalQuery(s) | AvgQuery(ms)")
    print(f"---------------|------------|---------------|-------------")
    print(f"Standard       | {recall_std:<10.4f} | {tot_time_std:<13.2f} | {avg_time_std:<11.2f}")
    # Only show PCA line if PCA was actually attempted
    if idx_pca.pca_yes:
        print(f"PCA-optimized  | {recall_pca:<10.4f} | {tot_time_pca:<13.2f} | {avg_time_pca:<11.2f}")

Selecting query data...
Using 40000 queries.
Computing ground truth...


INFO:__main__:Initialized HNSW (dim=100, PCA=off, PCA components=N/A)


Ground truth computed in 9.37s

Building standard HNSW index (no PCA)...


Indexing standard HNSW: 100%|██████████| 400000/400000 [47:04<00:00, 141.60it/s]  
INFO:__main__:Initialized HNSW (dim=100, PCA=on (layers >= 2), PCA components=25)


Standard build time: 2307.32s
Standard index height: 4

Building PCA-enabled HNSW index...


...

PCA-enabled index build time (before finalize): 2317.24s
PCA-enabled index height: 5

Finalizing PCA models on index...
PCA finalize time: 0.00s

Precomputing PCA projections for queries...
Skipping query PCA precomputation as PCA was not applied to any layers.
Query PCA precomputation time: 0.0000s

Evaluating standard search...


Standard query: 100%|██████████| 40000/40000 [03:58<00:00, 167.62it/s]



Evaluating PCA-optimized search...


PCA-opt query: 100%|██████████| 40000/40000 [03:47<00:00, 175.70it/s]


--- Results ---
Parameters: M=20, ef_construction=300, k=100
PCA Parameters: components=25, applied_to_layers=None

Build Times:
Standard Build: 2307.32s
PCA Build (Index): 2317.24s
PCA Build (Finalize): 0.00s
Total PCA Build: 2317.24s

Standard: recall@100=0.6762, total_query_time=238.64s, avg_query_time=5.90ms
PCA-opt:  recall@100=0.6546, total_query_time=227.66s, avg_query_time=5.64ms

--- Comparison Summary ---
Method         | Recall@100 | TotalQuery(s) | AvgQuery(ms)
---------------|------------|---------------|-------------
Standard       | 0.6762     | 238.64        | 5.90       
PCA-optimized  | 0.6546     | 227.66        | 5.64       


In [9]:
if __name__ == "__main__":
    

    # evaluate standard search
    print("Evaluating standard search…")
    total_r_std, times_std = 0, []
    tstart = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="Standard query")):
        ts = time.perf_counter()
        out = idx_std.search(q, k)
        te = time.perf_counter()
        times_std.append(te - ts)
        total_r_std += recall_at_k(gt_i[i], out)
    tot_time_std = time.perf_counter() - tstart
    avg_time_std = np.mean(times_std) * 1000
    recall_std = total_r_std / num_queries

    # evaluate PCA-optimized search
    print("Evaluating PCA-optimized search…")
    total_r_pca, times_pca = 0, []
    tstart = time.perf_counter()
    for i, q in enumerate(tqdm(queries, desc="PCA-opt query")):
        ts = time.perf_counter()
        out = idx_pca.search_pca_opt(i, q, pca_q, k)
        te = time.perf_counter()
        times_pca.append(te - ts)
        total_r_pca += recall_at_k(gt_i[i], out)
    tot_time_pca = time.perf_counter() - tstart
    avg_time_pca = np.mean(times_pca) * 1000
    recall_pca = total_r_pca / num_queries

    # output results
    print(f"Standard: recall={recall_std:.4f}, total_query_time={tot_time_std:.2f}s, avg_query_time={avg_time_std:.2f}ms")
    print(f"PCA-opt: recall={recall_pca:.4f}, total_query_time={tot_time_pca:.2f}s, avg_query_time={avg_time_pca:.2f}ms")

    print("\n--- Comparison Summary ---")
    print(f"Method         | Recall@{k} | TotalQuery(s) | AvgQuery(ms)")
    print(f"Standard       | {recall_std:.4f}    | {tot_time_std:.2f}       | {avg_time_std:.2f}")
    print(f"PCA-optimized  | {recall_pca:.4f}    | {tot_time_pca:.2f}       | {avg_time_pca:.2f}")

Evaluating standard search…


Standard query:   0%|          | 0/40000 [00:00<?, ?it/s]


TypeError: recall_at_k() missing 1 required positional argument: 'k_val'

In [ ]:
print("Height of HNSW graph:", idx_pca.entry_point_level)

Height of HNSW graph: 5
